In [2]:
import torch 
from torch import nn

# class DetectorPanel(nn.Module):
#     def __init__(self, xy, z, zfromhodoscope):
#         super().__init__()
#         self.xy = xy
#         self.z = z
#         self.zfromhodoscope = zfromhodoscope

#     def globalz(self):
#         return self.z+self.zfromhodoscope
    
#     def print(self):
#         #print(f"\t Panel has xy={self.xy.data}, z={self.z.data+self.globalz.data}")
#         print(f"\t Panel has xy={self.xy.data}, relative z={self.z.data}, z from hodoscope={self.zfromhodoscope.data}, global z={self.globalz()}")
 
# class Hodoscope(nn.Module):
#     def __init__(self, xy, z):
#         super().__init__()
#         self.xy = nn.parameter.Parameter(data=xy, requires_grad=True)
#         self.z = nn.parameter.Parameter(data=z, requires_grad=True)
#         self.hodoscopes = []
        
#     def generatehodoscopes(self, relzs):
#         self.positionList= [nn.parameter.Parameter(data=i, requires_grad=False) for i in relzs]
#         for z in self.positionList:
#             self.hodoscopes.append(DetectorPanel(self.xy,z, self.z))

#     def print(self):
#         print(f"Hodoscope has xy={self.xy.data}, z={self.z.data}")
            
#     def printhodoscopes(self):
#         for panel in self.panels:
#             panel.print()
            
            
#     def modifyPosition(self, xy, z):
#         #self.xy = nn.parameter.Parameter(xy)
#         #self.z = nn.parameter.Parameter(z)
#         self.xy.data = xy
#         self.z.data = z

    

# # hod = Hodoscope(torch.Tensor([2,3]), torch.Tensor([5]))
# # positionList=torch.Tensor([0.,0.3,0.9])

# # hod.generatePanels(positionList)
# # hod.print()

# # hod.printPanels()

# # hod.modifyPosition(torch.Tensor([1,2]), torch.Tensor([8]))

# # hod.print()
# # hod.printPanels()

## Layers and hodoscopes


The default *layer* class `PanelDetectorLayer` in TomOpt uses multiple *panels* `DetectorPanel` to record muon position.

`PanelDetectorLayer` inherits from the `AbsDetectorLayer` and **MUST** provide the following **methods**:

 - `forward`: The forward method to propagate the muons. It is a central aspect of `nn.Module`.
 - `get_cost`
 - `conform_detector`
  - `assign_budget`

and the following **features**:

 - `pos`: the position of the layer, either `above` or `bellow`. 
 - `lw`: the length and width of the layer.
 - `z`: the z position of the top of gthe layer. 
 - `size`: the heigth of the layer such that z - size is the bottom of the layer.
 - `device`: torch.device.

In [59]:
from tomopt.volume.layer import AbsDetectorLayer
from tomopt.volume.panel import DetectorPanel
from typing import List, Union, Optional, Iterator, Tuple
from torch import Tensor
import numpy as np
from tomopt.muon import MuonBatch

from tomopt.core import DEVICE

class Hodoscope(nn.Module):

    def __init__(self,
                 *,
                 init_xyz: Tuple[float, float, float],
                 init_xyz_span: Tuple[float, float, float],
                 n_panels: int = 3,
                 dz_gap: float = 0.97,
                 res: float = 1000,
                 eff: float = 0.9,
                 m2_cost: float = 1.,
                 budget: Optional[Tensor] = None,
                 realistic_validation: bool = True,
                 device: torch.device = DEVICE):
        
        super().__init__()
        self.realistic_validation, self.device = realistic_validation, device
        # self.register_buffer("m2_cost", torch.tensor(float(m2_cost), device=self.device))
        self.xy = nn.Parameter(torch.tensor(init_xyz[:2], device=self.device))
        self.z = nn.Parameter(torch.tensor(init_xyz[-1], device=self.device))
        self.xyz_span = nn.Parameter(torch.tensor(init_xyz_span, device=self.device))
        self.n_panels = n_panels
        self.dz_gap = dz_gap
        self.res = res
        self.eff = eff
        self.panels = self.generate_init_panels()
        self.device = device

    def __repr__(self) -> str:
        return f"""Hodoscope located at z={self.z}"""

    
    def generate_init_panels(self) -> List[DetectorPanel]:

        return nn.ModuleList([DetectorPanel(res=self.res, eff=self.eff,
                    init_xyz=[self.xy[0], self.xy[1], 
                    self.z-self.dz_gap*self.xyz_span[2]-
                    (self.xyz_span[2]-2*self.dz_gap)*i/(self.n_panels-1)], 
                    init_xy_span=[1.0, 1.0],
                    device=DEVICE) for i in range(self.n_panels)])

    def get_cost(self) -> Tensor:

        return torch.sum([p.get_cost() for p in self.panels])

class HodoscopeDetectorLayer(AbsDetectorLayer):

    def __init__(self, 
                 pos:str, 
                 *,
                 lw:Tensor,
                 z:float,
                 size:float, 
                 hodoscopes: List[Hodoscope],
    ):
        if isinstance(hodoscopes, list):
            hodoscopes = nn.ModuleList(hodoscopes)

        super().__init__(pos=pos, lw=lw, z=z, size=size, device=self.get_device(hodoscopes))
        self.hodoscopes = hodoscopes

        if isinstance(hodoscopes[0], Hodoscope):
            self.type_label = "hodoscope"
            self._n_costs = len(self.hodoscopes)

    @staticmethod
    def get_device(hodoscopes: nn.ModuleList) -> torch.device:

        r"""
        Helper method to ensure that all panels are on the same device, and return that device.
        If not all the panels are on the same device, then an exception will be raised.

        Arguments:
            panels: ModuleLists of either :class:`~tomopt.volume.panel.DetectorPanel` or :class:`~tomopt.volume.heatmap.DetectorHeatMap` objects on device

        Returns:
            Device on which all the panels are.
        """

        device = hodoscopes[0].device
        if len(hodoscopes) > 1:
            for h in hodoscopes[1:]:
                if h.device != device:
                    raise ValueError("All hodoscopes must use the same device, but found multiple devices")
        return device
    
    def get_panel_zorder(self) -> List[int]:
        r"""
        Returns:
            The indices of the panels in order of decreasing z-position.
        """
        
        z_order_panels=[]

        for h in self.hodoscopes:
            for p in h.panels:
                z_order_panels.append(p.z.detach().cpu().item())

        return list(np.argsort(z_order_panels))
    
    def yield_zordered_panels(self) -> Iterator[Tuple[int, DetectorPanel]]:
        r"""
        Yields the index of the panel, and the panel, in order of decreasing z-position.

        Returns:
            Iterator yielding panel indices and panels in order of decreasing z-position.
        """

        for i in self.get_panel_zorder():
             yield i, self.panels[i]

    def forward(self, mu: MuonBatch) -> None:
        r"""
        Propagates muons to each detector panel, in order of decreasing z-position, and calls their `get_hits` method to record hits to the muon batch.
        After this, the muons will be propagated to the bottom of the detector layer.

        Arguments:
            mu: the incoming batch of muons
        """

        for i, p in self.yield_zordered_panels():
            mu.propagate_dz(mu.z - p.z.detach())  # Move to panel
            hits = p.get_hits(mu)
            mu.append_hits(hits, self.pos)
        mu.propagate_dz(mu.z - (self.z - self.size))  # Move to bottom of layer

    def get_cost(self) -> Tensor:
        r"""
        Returns the total, current cost of the detector(s) in the layer, as computed by looping over the hodoscopes and summing the returned values of calls to
        their `get_cost` methods.

        Returns:
            Single-element tensor with the current total cost of the detector in the layer.
        """

        cost = None
        for p in self.panels:
            cost = p.get_cost() if cost is None else cost + p.get_cost()
        return cost

In [60]:
hod1 = Hodoscope(init_xyz= [0.25, 0.25, 0.25], init_xyz_span= [1., 1., 1.])

hod2 = Hodoscope(init_xyz= [1.25, 1.25, 1.25], init_xyz_span= [1., 1., 1.])

hods = [hod1, hod2]

In [61]:
hod_detector_above = HodoscopeDetectorLayer(pos = 'above', lw = Tensor([1,1]), z = 1., size = 0.5, 
                                            hodoscopes = hods)

hod_detector_above

HodoscopeDetectorLayer(
  (hodoscopes): ModuleList(
    (0): Hodoscope located at z=Parameter containing:
    tensor(0.2500, requires_grad=True)
    (1): Hodoscope located at z=Parameter containing:
    tensor(1.2500, requires_grad=True)
  )
)

In [41]:

import time as t

liste = []
t0 = t.time()
for i in range(100000):

    liste.append(i)

t.time()-t0

0.024484872817993164

In [42]:
t0 = t.time()
[i for i in range(100000)]

t.time()-t0

0.012005090713500977

In [52]:
dz = 1
dz_gap = 0.01
n_panels = 3
z = 1

a = [z-dz_gap*dz-(dz-2*dz_gap)*i/(n_panels-1) for i in range(n_panels)]
a

[0.99, 0.5, 0.010000000000000009]